In [1]:
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [2]:
# Essa função irá trazer 3 valores pra mim: os dados de treino, de teste e os dados propriamente dito.
def get_train_test(url, split_percent=0.8): # split_percent define a proporção das amostras de treino/teste
    df = read_csv(url, usecols=[1], engine='python')    #colocar em dataframe os dados
    data = np.array(df.values.astype("float32"))        #pegar os valores deles e colocar no tipo float
    scaler = MinMaxScaler(feature_range=(0,1))          # normalizar esses dados
    data = scaler.fit_transform(data).flatten()         
    n=len(data)
    # Aqui vai ocorrer a divisão do dataset em treino e teste
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data


In [3]:
# Função para preparar os input (variáveis independentes (X) e dependente (Y))
def get_XY(dat, time_steps):
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))
    return X, Y

In [5]:
# Função para criar o modelo de deep learning RNN
# Basicamente, pelo .add, serão adicionadas 2 camadas:
# 1) uma SimpleRNN, que vai fazer o papel recursivo, recebendo quantas camadas de RNN ele vai ter, o
#  tamanho do shape dos dados que to passando e o tipo de ativação.
# 2) uma camada densa (de saída), que vai depender tambem do quanto especifique de dense_units e do tipo de ativação
# No compilador, eu vou usar o erro quadrado médio como função de custo e o otimizador padrão Adam
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape = input_shape, activation = activation[0]))
    model.add(Dense(units = dense_units, activation = activation[1])) #ou seja, dentro da lista acvation, o 1°
    model.compile(loss="mean_squared_error", optimizer = "adam")      #vai pra Simple e o 2° vai pra dense
    return model

In [6]:
# Função auxiliar que vai printar o erro, por meio do erro quadrado médio.
def print_error(trainY, testY, train_predict, test_predict):
    # Erro das predições
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    # Print do RMSE
    print("Train RMSE: %.3f RMSE" % (train_rmse))
    print("Test RMSE: %.3f RMSE" % (test_rmse))

# Função auxiliar pra plotar os resultados
def plot_result(trainY, testY, train_predict, test_predict):
    actual = np.append(trainY, testY)
    predictions = np.append(train_predict, test_predict)
    rows = len(actual)
    plt.figure(figsize=(15,6), dpi=80)
    plt.plot(range(rows), actual)
    plt.plot(range(rows), predictions)
    plt.axvline(x=len(trainY), color="r")
    plt.legend(['Actual', "Predictions"])
    plt.xlabel("Observation number after given time steps")
    plt.ylabel("Sunspots scaled")
    plt.title("Actual and Predicted Values. The Red Line Separates The Training and Test Examples")

In [8]:
# Aqui começa o trabalho. Trazer a base de dados
sunspots_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv"

In [9]:
time_steps = 12

In [10]:
#aqui abaixo é o retorno da função criada lá em cima, em que informo apenas a URL e ele devole essas 3 info
train_data, test_data, data = get_train_test(sunspots_url) 

In [12]:
#pegando os dados de treino e teste de X e Y 
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

In [ ]:
#Criando o modelo e treinando ele
# Serão 3 camadas ocultas
# a camada densa (de saída) será 1 só
# shape (12,1)
# função de ativação (activation 0 e 1) são do tipo tangente hiperbólica
# serão 20 épocas: é a etapa de treinamento e validação dos dados do meu modelo de rede neural. E
# cada época ele vai treinando novamente. Como é uma recde recorrente, o output não é só a saída simplesmente, mas
# também 

